使用官方baseline：[https://aistudio.baidu.com/aistudio/projectdetail/1616829](https://aistudio.baidu.com/aistudio/projectdetail/1616829)

该示例展示了如何使用PaddleNLP快速实现[LIC2021机器阅读理解比赛](https://aistudio.baidu.com/aistudio/competition/detail/66)基线并进阶优化基线。

<br>

机器阅读理解 (Machine Reading Comprehension) 是指让机器阅读文本，然后回答和阅读内容相关的问题。阅读理解是自然语言处理和人工智能领域的重要前沿课题，对于提升机器的智能水平、使机器具有持续知识获取的能力等具有重要价值，近年来受到学术界和工业界的广泛关注。

自然语言理解对机器学习模型各方面的能力均有极高的要求。然而，当前的机器阅读理解数据集大多都只采用单一的指标来评测模型的好坏，缺乏对模型语言理解能力的细粒度、多维度评测，导致模型的具体缺陷很难被发现和改进。为了解决这个问题，中国计算机学会、中国中文信息学会和百度公司建立了细粒度的、多维度的评测数据集，从词汇理解、短语理解、语义角色理解、逻辑推理等多个维度检测模型的不足之处，从而推动阅读理解评测进入“精细化“时代。该数据集中的样本均来自于实际的应用场景，难度大，考察点丰富，覆盖了真实应用中诸多难以解决的问题。

DuReader<sub>checklist</sub>数据集包含训练集、开发集以及测试集。其中开发集和测试集中，既包含和训练集同分布的in-domain样本，也包含了按照checklist体系分类后的样本。对于一个给定的问题q、一个篇章p及其标题t，系统需要根据篇章内容，判断该篇章p中是否包含给定问题的答案，如果是，则给出该问题的答案a；否则输出“no answer”。数据集中的每个样本，是一个四元组<q, p, t, a>，例如：

* * *

**问题 q**: 番石榴汁热量

**篇章 p**: 番石榴性温,味甜、酸、涩…，最重要的是番石榴所含的脂肪热量较低,一个番石榴所含的脂肪约0.9克重或84卡路里。比起苹果,番石榴所含有的脂肪少38%,卡路里少42%。

**标题 t**: 番石榴汁的热量 - 妈妈网百科


**参考答案 a**: [‘一个番石榴所含的脂肪约0.9克重或84卡路里’]

* * *

**问题 q**: 云南文山市多少人口?

**篇章 p**: 云南省下辖8个市、8个少数民族自治州,面积39万平方千米,总人口4596万人,云南汉族人口为3062.9万人,占云南省总人口的66.63%...

**标题 t**: 云南总人口数多少人,2019年云南人口数量统计(最新)


**参考答案 a**: [‘无答案’]

* * *

DuReader<sub>checklist</sub>数据集旨在通过建立checklist评测体系，系统性地评估当前模型能力的不足之处。目前checklist体系中涉及到的自然语言理解能力包含：词汇理解、短语理解、语义角色理解以及推理能力等等。具体的分类体系可参考下图：
![checklist_framwork](https://ai-studio-static-online.cdn.bcebos.com/a27d5c7445884681b0d65be4eec1e6f3272aae1b4c534e178dfe2ec28a6ed302)

<br>



## 安装说明

* PaddlePaddle 安装

   本项目依赖于 PaddlePaddle 2.0 及以上版本，请参考 [安装指南](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/pip/windows-pip.html) 进行安装

* PaddleNLP 安装

   ```shell
   pip install --upgrade paddlenlp -i https://pypi.org/simple
   ```

* 环境依赖

    Python的版本要求 3.6+


AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装

In [1]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

     |████████████████████████████████| 440kB 9.6kB/s eta 0:00:01
     |████████████████████████████████| 112kB 16kB/s eta 0:00:01
     |████████████████████████████████| 92kB 9.6kB/s eta 0:00:01
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.20.3 which is incompatible.
  Found existing installation: dill 0.3.3
    Uninstalling dill-0.3.3:
      Successfully uninstalled dill-0.3.3
  Found existing installation: paddlenlp 2.0.0rc7
    Uninstalling paddlenlp-2.0.0rc7:
      Successfully uninstalled paddlenlp-2.0.0rc7


## 下载数据文件
在运行基线之前，需要下载DuReader<sub>checklist</sub>数据集

执行以下脚本，数据集会被保存到```dataset/```文件夹中。此外，基于[ERNIE-1.0](https://arxiv.org/abs/1904.09223)微调后的基线模型参数也会被保存在`finetuned_model/ `文件夹中，可供直接预测使用。

In [2]:
#!sh download.sh

Download DuReader-checklist dataset
--2021-07-16 11:04:39--  https://dataset-bj.cdn.bcebos.com/lic2021/dureader_checklist.dataset.tar.gz
Resolving dataset-bj.cdn.bcebos.com (dataset-bj.cdn.bcebos.com)... 182.61.128.166
Connecting to dataset-bj.cdn.bcebos.com (dataset-bj.cdn.bcebos.com)|182.61.128.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1636803 (1.6M) [application/x-gzip]
Saving to: ‘dureader_checklist.dataset.tar.gz’

dureader_checklist. 100%[===================>]   1.56M  --.-KB/s    in 0.02s   

2021-07-16 11:04:39 (69.4 MB/s) - ‘dureader_checklist.dataset.tar.gz’ saved [1636803/1636803]

dataset/
dataset/dev.json
dataset/train.json
dataset/License.docx
Download fine-tuned parameters
--2021-07-16 11:04:39--  https://dataset-bj.cdn.bcebos.com/lic2021/dureader_checklist.finetuned_model.tar.gz
Resolving dataset-bj.cdn.bcebos.com (dataset-bj.cdn.bcebos.com)... 182.61.128.166
Connecting to dataset-bj.cdn.bcebos.com (dataset-bj.cdn.bcebos.com)|182.61.1

# 数据准备

数据准备流程如下：

![](https://ai-studio-static-online.cdn.bcebos.com/89ba02da6f914297ae2fc438d1c9f773556f226652134fb684ac0186bfa5bb7d)



## 使用`load_dataset()`自定义数据集

DuReader<sub>checklist</sub>数据集采用SQuAD数据格式，我们可以写出数据文件读取函数，传入`load_dataset()`。即可创建数据集。

In [14]:
from paddlenlp.datasets import load_dataset
import json

# 根据本地文件格式定义数据读取生成器
def read(filename):
    with open(filename, "r", encoding="utf8") as f:
        input_data = json.load(f)
        for entry in input_data['data']:
            title = entry.get("title", "").strip()
            for paragraph in entry["paragraphs"]:
                context = paragraph["context"].strip()
                for qa in paragraph["qas"]:
                    qas_id = qa["id"]
                    question = qa["question"].strip()
                    answer_starts = []
                    answers = []
                    is_impossible = False

                    if "is_impossible" in qa.keys():
                        is_impossible = qa["is_impossible"]

                    answer_starts = [
                        answer["answer_start"] for answer in qa.get("answers",[])
                    ]
                    answers = [
                        answer["text"].strip() for answer in qa.get("answers",[])
                    ]

                    yield {
                        'id': qas_id,
                        'title': title,
                        'context': context,
                        'question': question,
                        'answers': answers,
                        'answer_starts': answer_starts,
                        'is_impossible': is_impossible
                    }

# 将生成器传入load_dataset
train_ds = load_dataset(read, filename='dataset/train.json', lazy=False)
dev_ds = load_dataset(read, filename='dataset/dev.json', lazy=False)

for idx in range(2):
    print(train_ds[idx]['question'])
    print(train_ds[idx]['context'])
    print(train_ds[idx]['answers'])
    print(train_ds[idx]['answer_starts'])
    print(train_ds[idx]['is_impossible'])
    print()

属鼠小名多米的寓意
——使用叠字的小名是很好听的,对于刚刚出生的男孩来说,这样的名字是很方便记忆的,这也就拉近了孩子与家长之间的距离。糕,代表的是“用面粉所制成的食品”,那么此字在含义上是带有“多米”的含义,作为属鼠男孩的小名,寓意上是很不错的。
['']
[-1]
True

红烧肉煮鲍鱼的做法
将锅中倒入油,小火爆香蒜末,一片姜的姜末,倒入上一步骤中蒸鲍鱼的汤汁,倒入蚝油,鲍鱼汁,蒸鱼豉油,东古一品鲜,味极鲜,糖,鸡精,鸡粉调味。如果汤汁少,放入少许水。待汤汁快要沸腾时,放入鲍鱼,均匀的裹上汤汁。最后,将鲍鱼放入已经摆盘的西兰花上面,将汤汁淋到鲍鱼上面就大功告成啦🤓
['']
[-1]
True



关于更多自定义数据集相关内容，请移步[如何自定义数据集](https://paddlenlp.readthedocs.io/en/latest/data_prepare/dataset_self_defined.html)

## 使用 `paddlenlp.transformers.ErnieTokenizer`将数据转为Feature

DuReader<sub>checklist</sub>数据集采用SQuAD数据格式，InputFeature使用滑动窗口的方法生成，即一个example可能对应多个InputFeature。

由于文章加问题的文本长度可能大于max_seq_length，答案出现的位置有可能出现在文章最后，所以不能简单的对文章进行截断。

那么对于过长的文章，则采用滑动窗口将文章分成多段，分别与问题组合。再用对应的tokenizer转化为模型可接受的feature。doc_stride参数就是每次滑动的距离。滑动窗口生成InputFeature的过程如下图：

![](https://ai-studio-static-online.cdn.bcebos.com/5776cf9ec00546bca047a0930c8f56a8b64d723e0ff04f269334522954bb7d90)

本基线中，我们使用的预训练模型是ERNIE，ERNIE对中文数据的处理是以字为单位。**PaddleNLP对于各种预训练模型已经内置了相应的tokenizer**，指定想要使用的模型名字即可加载对应的tokenizer。

tokenizer的作用是将原始输入文本转化成模型可以接受的输入数据形式。

In [15]:
import paddlenlp

# 设置模型名称
MODEL_NAME = 'ernie-1.0'
tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)

[2021-07-16 11:27:42,405] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


## 调用`map()`方法批量处理数据

由于我们传入了`lazy=False`，所以我们使用`load_dataset()`自定义的数据集是`MapDataset`对象，`MapDataset`是`paddle.io.Dataset`的功能增强版本。其内置的`map()`方法适合用来进行批量数据集处理。`map()`方法传入的是一个用于数据处理的function。正好可以与tokenizer相配合。

以下是本基线中的用法：

In [5]:
from src.utils import prepare_train_features, prepare_validation_features
from functools import partial

max_seq_length = 512
doc_stride = 128

train_trans_func = partial(prepare_train_features, 
                           max_seq_length=max_seq_length, 
                           doc_stride=doc_stride,
                           tokenizer=tokenizer)

train_ds.map(train_trans_func, batched=True)

dev_trans_func = partial(prepare_validation_features, 
                           max_seq_length=max_seq_length, 
                           doc_stride=doc_stride,
                           tokenizer=tokenizer)
                           
dev_ds.map(dev_trans_func, batched=True)

In [6]:
for idx in range(2):
    print(train_ds[idx]['input_ids'])
    print(train_ds[idx]['token_type_ids'])
    print(train_ds[idx]['overflow_to_sample'])
    print(train_ds[idx]['offset_mapping'])
    print(train_ds[idx]['start_positions'])
    print(train_ds[idx]['end_positions'])
    print(train_ds[idx]['answerable_label'])
    print()

从以上结果可以看出，数据集中的example已经被转换成了模型可以接收的feature，包括input_ids、token_type_ids、答案的起始位置等信息。
其中：

* `input_ids`: 表示输入文本的token ID。
* `token_type_ids`: 表示对应的token属于输入的问题还是答案。（Transformer类预训练模型支持单句以及句对输入）。
* `overflow_to_sample`: feature对应的example的编号。
* `offset_mapping`: 每个token的起始字符和结束字符在原文中对应的index（用于生成答案文本）。
* `start_positions`: 答案在这个feature中的开始位置。
* `end_positions`: 答案在这个feature中的结束位置。
* `answerable_label`: 答案在这个feature中是否存在，存在为1，不存在为0。

关于本基线中数据处理的详细过程请参见`src/utils.py`。

更多有关数据处理的内容，请移步[数据处理](https://paddlenlp.readthedocs.io/en/latest/data_prepare/data_preprocess.html)。

## Batchify和数据读入

使用`paddle.io.BatchSampler`和`paddlenlp.data`中提供的方法把数据组成batch。

然后使用`paddle.io.DataLoader`接口多线程异步加载数据。

`batchify_fn`详解：

![](https://ai-studio-static-online.cdn.bcebos.com/30e43d4659384375a2a2c1b890ca5a995c4324d7168e49cebf1d2a1e99161f7d)


In [7]:
import paddle
from paddlenlp.data import Stack, Dict, Pad

batch_size = 2

# 初始化BatchSampler
train_batch_sampler = paddle.io.BatchSampler(
    train_ds, batch_size=batch_size, shuffle=True)

dev_batch_sampler = paddle.io.BatchSampler(
    dev_ds, batch_size=batch_size, shuffle=False)

# 定义batchify_fn
train_batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id), 
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    "start_positions": Stack(dtype="int64"),  
    "end_positions": Stack(dtype="int64"),  
    "answerable_label": Stack(dtype="int64")  
}): fn(samples)

dev_batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id), 
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id)
}): fn(samples)

# 初始化DataLoader
train_data_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_sampler=train_batch_sampler,
    collate_fn=train_batchify_fn,
    return_list=True)

dev_data_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_sampler=dev_batch_sampler,
    collate_fn=dev_batchify_fn,
    return_list=True)

# 组网训练

## 使用PaddleNLP一键加载预训练模型

以下项目以ERNIE为例，介绍如何将预训练模型Fine-tune完成DuReader<sub>checklist</sub>阅读理解任务。

DuReader<sub>checklist</sub>阅读理解任务的本质是答案抽取任务和句对分类任务。根据输入的问题和文章，预测答案在文章中的起始位置和结束位置以及判断答案是否存在。原理如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/1592580788c54e7b9b2491862836962c4163fe66c8bc44aabada0cf89f56f0e7)


In [8]:
from src.models import ErnieForQuestionAnswering

model = ErnieForQuestionAnswering.from_pretrained(MODEL_NAME)

### 设计loss function

ErineForQuestionAnswering模型对将BertModel的sequence_output拆开成start_logits和end_logits输出，并将pooled_output作为cls_logits输出，所以DuReader<sub>checklist</sub>的loss由start_loss和end_loss和cls_logits三部分组成，我们需要自己定义loss function。

对于答案起始位置和结束位置和答案是否存在的预测可以分别看成三个分类任务。所以设计的loss function如下：

In [9]:
class CrossEntropyLossForChecklist(paddle.nn.Layer):
    def __init__(self):
        super(CrossEntropyLossForChecklist, self).__init__()
        
    def forward(self, y, label):
        start_logits, end_logits, cls_logits = y
        start_position, end_position, answerable_label = label
        start_position = paddle.unsqueeze(start_position, axis=-1)
        end_position = paddle.unsqueeze(end_position, axis=-1)
        answerable_label = paddle.unsqueeze(answerable_label, axis=-1)
        start_loss = paddle.nn.functional.softmax_with_cross_entropy(
            logits=start_logits, label=start_position, soft_label=False)
        start_loss = paddle.mean(start_loss)
        end_loss = paddle.nn.functional.softmax_with_cross_entropy(
            logits=end_logits, label=end_position, soft_label=False)
        end_loss = paddle.mean(end_loss)
        cls_loss = paddle.nn.functional.softmax_with_cross_entropy(
            logits=cls_logits, label=answerable_label, soft_label=False)
        cls_loss = paddle.mean(cls_loss)
        mrc_loss = (start_loss + end_loss) / 2
        loss = (mrc_loss + cls_loss) /2
        return loss

## 模型配置

### 配置优化策略
适用于ERNIE/BERT这类Transformer模型的学习率为warmup的动态学习率。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2bc624280a614a80b5449773192be460f195b13af89e4e5cbaf62bf6ac16de2c" width="40%" height="30%"/> <br />
</p>
<br><center>图3：动态学习率示意图</center></br>

In [10]:
# 训练过程中的最大学习率
learning_rate = 3e-5 

# 训练轮次
epochs = 2

# 学习率预热比例
warmup_proportion = 0.1

# 权重衰减系数，类似模型正则项策略，避免模型过拟合
weight_decay = 0.01

num_training_steps = len(train_data_loader) * epochs

# 学习率衰减策略
lr_scheduler = paddlenlp.transformers.LinearDecayWithWarmup(learning_rate, num_training_steps,
                                    warmup_proportion)

decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义优化器
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

## 模型训练

模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch后，程序通过evaluate()调用`compute_prediction_checklist()`生成可提交的答案。

In [11]:
from src.utils import evaluate

criterion = CrossEntropyLossForChecklist()
global_step = 0
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        
        global_step += 1
        input_ids, token_type_ids, start_positions, end_positions, answerable_label = batch
        logits = model(input_ids=input_ids, token_type_ids=token_type_ids)
        loss = criterion(logits, (start_positions, end_positions,answerable_label))

        if global_step % 100 == 0 :
            print("global step %d, epoch: %d, batch: %d, loss: %.5f" % (global_step, epoch, step, loss))
            
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()

evaluate(model, dev_data_loader)

## 模型评估

本次评测所采用的评价指标为F1-score和Exact Match (EM)。其中F1作为主要评价指标，EM作为辅助评价指标。

评估脚本的运行参考以下命令:

```
sh run_eval.sh dataset_file pred_file
```

其中`dataset_file `是数据集文件，`pred_file`是模型预测结果，例如

```
sh run_eval.sh dataset/dev.json prediction.json
```

In [12]:
!sh run_eval.sh dataset/dev.json prediction.json

# 如何提升结果

## 数据增强

PaddleNLP内置的Dureader-robust数据集与该比赛的数据集格式相似。可以通过PaddleNLP内置的`load_dataset()`方法一键加载数据集用于数据集增强（需要进行一些处理保证格式一致）

In [13]:
from paddlenlp.datasets import load_dataset

train_robust, dev_robust = load_dataset('dureader_robust',splits=('train','dev'))

for idx in range(2):
    print(train_robust[idx]['question'])
    print(train_robust[idx]['context'])
    print(train_robust[idx]['answers'])
    print(train_robust[idx]['answer_starts'])
    print()

## 使用更大的模型

PaddleNLP不仅支持ERNIE预训练模型，还支持BERT、RoBERTa等预训练模型，加载方式都与ERNIE的加载方式相同。
使用更大的模型（例如`roberta-wwm-ext-large`）通常可以获得更好的效果。

更多预训练模型参考：[https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/transformers.md](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/transformers.md)

## 对抗训练

通过在词向量中添加干扰词向量来生成干扰样本。

重新训练模型，以增强模型的鲁棒性。


## 知识蒸馏，模型融合等

以上基线实现基于PaddleNLP，开源不易，希望大家多多支持~ 
**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
![](https://ai-studio-static-online.cdn.bcebos.com/a0e8ca7743ea4fe9aa741682a63e767f8c48dc55981f4e44a40e0e00d3ab369e)

**更多使用方法可参考PaddleNLP教程**

- [使用seq2vec模块进行句子情感分类](https://aistudio.baidu.com/aistudio/projectdetail/1283423)
- [使用预训练模型ERNIE优化情感分析](https://aistudio.baidu.com/aistudio/projectdetail/1294333)
- [使用BiGRU-CRF模型完成快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)
- [使用预训练模型ERNIE优化快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1329361)
- [使用Seq2Seq模型完成自动对联](https://aistudio.baidu.com/aistudio/projectdetail/1321118)
- [使用预训练模型ERNIE-GEN自动写诗](https://aistudio.baidu.com/aistudio/projectdetail/1339888)
- [使用TCN网络完成新冠疫情病例数预测](https://aistudio.baidu.com/aistudio/projectdetail/1290873)
- [自定义数据集实现文本多分类任务](https://aistudio.baidu.com/aistudio/projectdetail/1468469)

# 加入交流群，一起学习吧

现在就加入PaddleNLP的QQ技术交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >